# Option A - use FSL if Installed

In [1]:
import subprocess
import os

def mask_nifti(mask_path, image_path):
    # Extract directory and filename without extension
    image_dir, image_filename = os.path.split(image_path)
    image_basename, image_ext = os.path.splitext(image_filename)
    
    # Ensure the extension is .nii or .nii.gz
    if image_ext == '.gz':
        image_basename, _ = os.path.splitext(image_basename)
        image_ext = '.nii.gz'
    
    # Define the output path
    output_path = os.path.join(image_dir, f"{image_basename}_masked{image_ext}")
    
    # Build the FSL command
    fsl_command = ['fslmaths', image_path, '-mas', mask_path, output_path]
    
    # Execute the command
    result = subprocess.run(fsl_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    # Check if the command executed successfully
    if result.returncode != 0:
        # An error occurred, handle it here
        print(f"Error running fslmaths: {result.stderr.decode('utf-8')}")
        return None
    
    # Return the path to the masked image
    return output_path


In [2]:
mask_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/rimlf/sub-teke/atrophy/sub-teke_MNI152_T1_5_0mm_sphere_roi.nii.gz'
image_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/rimlf/sub-teke/atrophy/sub-TeKe_cerebrospinal_fluid_generated_nifti_wrp.nii'

In [3]:
masked_image_path = mask_nifti(mask_path=mask_path, image_path=image_path)
print(f"Masked image saved to: {masked_image_path}")

Error running fslmaths: 
          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!

Image Exception : #3 :: Attempted to multiply images of different sizes
libc++abi: terminating with uncaught exception of type std::runtime_error: Attempted to multiply images of different sizes
/Users/cu135/fsl/share/fsl/bin/fslmaths: line 2:  9060 Abort trap: 6           /Users/cu135/fsl/bin/fslmaths "$@"

Masked image saved to: None


# Option B - Pythonic Implementation

In [6]:
import nibabel as nib
# Load the input NIFTI file and the target NIFTI file
input_nifti = nib.load('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/rimlf/sub-teke/atrophy/sub-TeKe_cerebrospinal_fluid_generated_nifti_wrp.nii')
target_nifti = nib.load('/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/rimlf/sub-teke/atrophy/sub-teke_MNI152_T1_5_0mm_sphere_roi.nii.gz')
mni_mask_nifti = nib.load('/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii')

Generate Mask

In [5]:
import numpy as np
# Get the data arrays
target_data = target_nifti.get_fdata()

# Define the threshold for binarization
threshold = 0  # Replace with your desired threshold value

# Binarize the target data
binary_mask = (target_data > threshold).astype(np.float32)

# Ensure the dimensions of the input data and binary mask match
input_data = input_nifti.get_fdata()
if input_data.shape != binary_mask.shape:
    raise ValueError("The dimensions of the input NIFTI file and the target NIFTI file do not match.")
# Apply the binary mask to the input data
masked_data = input_data * binary_mask


NameError: name 'target_nifti' is not defined

Optionally Binarize the Masked Data

In [3]:
# masked_data = (masked_data > 0).astype(np.float32)

In [4]:

# Create a new NIFTI image with the masked data and the original affine
masked_img = nib.Nifti1Image(masked_data, input_nifti.affine, input_nifti.header)

# Save the masked image
nib.save(masked_img, '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook06/ongoing_ungodly_amount_of_edits/sub-sphere_boi/ses-01/roi/sub-sphere_boi_MNI152_T1_10.0mm_sphere_roiMASKEDBOI.nii.gz')
